In [1]:
from libcomcat.utils import get_phase_dataframe, get_summary_data_frame
from libcomcat.search import search,count,get_event_by_id
from datetime import datetime
import os.path
import pandas as pd

This will be an exploration of the libcomcat API, which provides a wrapper around the ComCat web API, which is documented here:

https://earthquake.usgs.gov/fdsnws/event/1/

The `search` function allows you to search for the summary versions of earthquakes.  This summary is described here:

https://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php

In [2]:
eventlist = search(starttime=datetime(1994,1,17,12,30),
                       endtime=datetime(1994,1,18,12,35),
                       minmagnitude=5.2)

The `search` function returns a list of SummaryEvent objects, which have a number of basic attributes:

In [3]:
event = eventlist[0]
print(event.id)
print(event.time)
print(event.latitude)
print(event.longitude)
print(event.depth)
print(event.magnitude)
print(event.location)
print(event.url)

ci3144585
1994-01-17 12:30:55.390000
34.213
-118.537
18.202
6.7
1km NNW of Reseda, CA
https://earthquake.usgs.gov/earthquakes/eventpage/ci3144585


You can also get a list of properties from the summary object, and access those properties using the [] operator.

In [4]:
print(event.properties)
print(event['gap'])

['nst', 'cdi', 'ids', 'mag', 'felt', 'tsunami', 'net', 'types', 'gap', 'time', 'status', 'dmin', 'alert', 'sources', 'code', 'detail', 'title', 'sig', 'place', 'url', 'rms', 'mmi', 'magType', 'tz', 'updated', 'type']
29.7


Summary objects also know what products (shakemap,pager, etc.) exist for this event.

In [5]:
event.hasProduct('shakemap')

True

To turn this list of objects into a useful dataframe catalog, use the `get_summary_data_frame()` function:

In [6]:
df = get_summary_data_frame(eventlist)
df

,id,time,location,latitude,longitude,depth,magnitude,url
0,ci3144585,1994-01-17 12:30:55.390,"1km NNW of Reseda, CA",34.213,-118.537,18.202,6.70,https://earthquake.usgs.gov/earthquakes/eventp...
1,ci3149646,1994-01-17 12:31:58.120,"1km ENE of Granada Hills, California",34.275,-118.493,5.317,5.89,https://earthquake.usgs.gov/earthquakes/eventp...
2,ci3144694,1994-01-17 12:40:36.120,"9km N of Chatsworth, California",34.340,-118.614,5.395,5.20,https://earthquake.usgs.gov/earthquakes/eventp...
3,ci3141273,1994-01-17 23:33:30.690,"7km NNE of Simi Valley, California",34.326,-118.698,9.083,5.58,https://earthquake.usgs.gov/earthquakes/eventp...
4,ci3141286,1994-01-18 00:43:08.890,"10km ESE of Piru, California",34.377,-118.698,10.709,5.24,https://earthquake.usgs.gov/earthquakes/eventp...
5,usp00066tf,1994-01-18 06:46:10.120,"Java, Indonesia",-8.364,113.187,140.700,5.20,https://earthquake.usgs.gov/earthquakes/eventp...


In addition to the summary version of an object, there is a DetailEvent object created from the JSON described here:

https://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson_detail.php

The detail object can be retrieved one of two ways:

 * the `getDetailEvent()` method of the SummaryEvent object
 * the `get_event_by_id()` function in the search module.

In [7]:
detail_event = event.getDetailEvent()
detail_event

ci3144585 1994-01-17 12:30:55.390000 (34.213,-118.537) 18.2 km M6.7

The DetailEvent object has many of the same attributes as the SummaryEvent object (id, time, latitude, etc.)  It also has a method to retrieve product information from the detailed JSON.

In [8]:
dyfi = detail_event.getProducts('dyfi')[0]

In [9]:
dyfi

Product dyfi from us updated 2015-01-28 02:28:32.677000 containing 22 content files.

Product objects have methods relating to their content files.  The primary one of interest is `getContent()`, which downloads content files from urls in the detailed JSON.  Here we will download the DYFI geocoded responses, and then load those into a Pandas dataframe.

In [10]:
filename = os.path.join(os.path.expanduser('~'),'cdi_geo.txt')
url = dyfi.getContent('cdi_geo.txt',filename=filename)

In [11]:
columns = ['Geocoded box','CDI','No. of responses',
           'Hypocentral distance','Latitude','Longitude',
           'Suspect?','City','State']
cdi_geo = pd.read_csv(filename,names=columns,skiprows=1)

In [12]:
cdi_geo.head()

,Geocoded box,CDI,No. of responses,Hypocentral distance,Latitude,Longitude,Suspect?,City,State
0,UTM:(11S 001 419 10000),2.2,1,533,37.7737,-122.5050,0,UTM:(11S 001 419 10000),NaN
1,UTM:(11S 001 420 10000),1.0,1,541,37.8634,-122.5118,0,UTM:(11S 001 420 10000),NaN
2,UTM:(11S 002 419 10000),2.0,2,527,37.7789,-122.3920,0,UTM:(11S 002 419 10000),NaN
3,UTM:(11S 003 419 10000),3.4,1,520,37.7840,-122.2790,0,UTM:(11S 003 419 10000),NaN
4,UTM:(11S 004 410 10000),3.8,1,446,36.9811,-122.1105,0,UTM:(11S 004 410 10000),NaN


We have a function that takes a DetailEvent object and returns a pandas DataFrame with phase data.

In [13]:
phases = get_phase_dataframe(detail_event)
phases.head()

,Channel,Distance,Azimuth,Phase,Arrival Time,Status,Residual,Weight,Agency
0,ZY.LA00.EHZ.,0.1267,147.3,P,1994-01-17 12:30:55.620,manual,None,None,CI
1,ZY.LA00.ELN.,0.1267,147.3,P,1994-01-17 12:30:55.630,manual,None,None,CI
2,ZY.LA00.ELE.,0.1267,147.3,S,1994-01-17 12:30:59.120,manual,None,None,CI
3,ZY.LA02.EHZ.,0.1792,146.7,P,1994-01-17 12:30:57.740,manual,None,None,CI
4,ZY.LA02.EHN.,0.1792,146.7,S,1994-01-17 12:30:59.770,manual,None,None,CI
